# Imports and installations

In [ ]:
!pip install cartopy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import  xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import math
from tqdm.auto import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

from cartopy.util import add_cyclic_point
from matplotlib import animation
from tqdm.auto import tqdm
from datetime import datetime,timedelta

from matplotlib.cm import get_cmap
warnings.filterwarnings("ignore")

In [ ]:
ds=xr.open_dataset('/content/drive/MyDrive/AP_HW/ERA5 Data/LS heat flux/lshf-2024.nc')
ds

# Anomaly



*   We calculated the **latent heat flux and sensible heat flux anomalies** for each event using the dates from `HW_list_rolling_def.csv` for **pre-monsoon (MAM)** and **monsoon (JJAS)** seasons.  
*   The **climatology** for **pre-monsoon (MAM)** and **monsoon (JJAS)** season was derived from ERA5 mean surface latent heat flux and mean surface sensible heat flux data (1981–2010) and computed in :
    - `climatology_calculation_heat_flux.ipynb`





## MAM

In [ ]:
# @title Heat date list
list_hw=pd.read_csv('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/HW_list_rolling_def.csv')
list_hw['Starting_Date'] = pd.to_datetime(list_hw["Starting_Date"])
list_hw['Ending_Date'] = pd.to_datetime(list_hw["Ending_Date"])

# separate hw for mam season
mam_hw_list =list_hw[(list_hw['Starting_Date'].dt.month >= 3) & (list_hw['Starting_Date'].dt.month <= 5)]
mam_hw_list.reset_index(inplace=True,drop=True)
mam_hw_list

In [ ]:
# @title Event data for composite analysis - LSHF anomaly

import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
import numpy as np
import matplotlib.colors as colors

# List to store anomalies for each event
lshf_anom_list = []

# Loop over heatwave events
for i in tqdm(range(len(mam_hw_list['Starting_Date']))):
    # Extract event year and dates
    year = mam_hw_list['Starting_Date'].dt.year.iloc[i]
    start_time = str(mam_hw_list['Starting_Date'][i])
    end_time = str(mam_hw_list['Ending_Date'][i])

    # Adjust times to Bangladesh timezone
    start_time = str(np.datetime64(start_time) - np.timedelta64(6, 'h'))  # Subtract 6h
    end_time = str(np.datetime64(end_time) - np.timedelta64(6, 'h') + np.timedelta64(24, 'h'))  # Include end date
    time_range = slice(start_time, end_time)
    event_no = i + 1

    ###################################
    ############# LSHF ################
    # Load LSHF and climatology
    ds_lshf = xr.open_dataset(f"/content/drive/MyDrive/AP_HW/ERA5 Data/LS heat flux/lshf-{year}.nc")
    lshf_clim = xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/hf/hf_climatology-mam.nc')

    # Select event period and compute mean
    ds_lshf = ds_lshf.sel(valid_time=time_range)
    ds_lshf = ds_lshf.mean(dim='valid_time', skipna=True)

    # Compute anomaly and assign event number
    lshf_anom = ds_lshf - lshf_clim
    lshf_anom = lshf_anom.assign_coords(event_no=event_no)

    # Add to anomaly list
    lshf_anom_list.append(lshf_anom)

# Concatenate all events
lshf_events_anom_mam = xr.concat(lshf_anom_list, dim='event_no')

# Save to NetCDF
lshf_events_anom_mam.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/hf/data/lshf_events_anom-mam.nc')


## JJAS

In [ ]:
# @title Heat date list
list_hw=pd.read_csv('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/HW_list_rolling_def.csv')
list_hw['Starting_Date'] = pd.to_datetime(list_hw["Starting_Date"])
list_hw['Ending_Date'] = pd.to_datetime(list_hw["Ending_Date"])

# separate hw for jjas season
jjas_hw_list =list_hw[(list_hw['Starting_Date'].dt.month >= 6) & (list_hw['Starting_Date'].dt.month <= 9)]
jjas_hw_list.reset_index(inplace=True,drop=True)
jjas_hw_list


In [ ]:
# @title Event data for composite analysis - LSHF anomaly (JJAS)

import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
import numpy as np
import matplotlib.colors as colors

# List to store anomalies for each event
lshf_anom_list = []

# Loop over JJAS heatwave events
for i in tqdm(range(len(jjas_hw_list['Starting_Date']))):
    # Extract event year and dates
    year = jjas_hw_list['Starting_Date'].dt.year.iloc[i]
    start_time = str(jjas_hw_list['Starting_Date'][i])
    end_time = str(jjas_hw_list['Ending_Date'][i])

    # Adjust times to Bangladesh timezone
    start_time = str(np.datetime64(start_time) - np.timedelta64(6, 'h'))  # Subtract 6h
    end_time = str(np.datetime64(end_time) - np.timedelta64(6, 'h') + np.timedelta64(24, 'h'))  # Include end date
    time_range = slice(start_time, end_time)
    event_no = i + 1

    ###################################
    ############# LSHF ################
    # Load LSHF and climatology
    ds_lshf = xr.open_dataset(f"/content/drive/MyDrive/AP_HW/ERA5 Data/LS heat flux/lshf-{year}.nc")
    lshf_clim = xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/hf/hf_climatology-jjas.nc')

    # Select event period and compute mean
    ds_lshf = ds_lshf.sel(valid_time=time_range)
    ds_lshf = ds_lshf.mean(dim='valid_time', skipna=True)

    # Compute anomaly and assign event number
    lshf_anom = ds_lshf - lshf_clim
    lshf_anom = lshf_anom.assign_coords(event_no=event_no)

    # Add to anomaly list
    lshf_anom_list.append(lshf_anom)

# Concatenate all events
lshf_events_anom_jjas = xr.concat(lshf_anom_list, dim='event_no')

# Save to NetCDF
lshf_events_anom_jjas.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/hf/data/lshf_events_anom-jjas.nc')


# Plot

We have plotted anomalies of **latent heat flux and sensible heat flux**.  

- Anomalies were calculated above

In [ ]:
levels=np.arange(-40, 40 + 1, 5)
projection=ccrs.PlateCarree()
fig,axes=plt.subplots(ncols=2,nrows=2,subplot_kw={'projection':projection},figsize=(10, 25))
# Data load
hf=xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/hf/data/lshf_events_anom-mam.nc')
# wind
# # Spatial region
lat_range=slice(27.5,20.00)
lon_range=slice(84,95.75)

hf=hf.sel(latitude=lat_range,longitude=lon_range)
hf_anom=hf.mean(dim='event_no',skipna=True)

latitude_hf=hf_anom['latitude'].values
longitude_hf=hf_anom['longitude'].values

# mslhf plot
ax=axes[0,1]
contourf=ax.contourf(longitude_hf,latitude_hf,hf_anom.mslhf,levels=levels,cmap='seismic',extend='both',transform=projection)
# cbar = plt.colorbar(contourf, ax=ax, orientation='horizontal', pad=0.01,fraction=0.06, label="${Pa}$")

ax.gridlines(draw_labels=['bottom'],visible=False)

ax.coastlines()
ax.set_title("(b) Mean surface latent heat flux - Pre-monsoon",fontsize=10)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.add_feature(cf.BORDERS.with_scale('10m'))
# msshf plot
ax=axes[0,0]
contourf=ax.contourf(longitude_hf,latitude_hf,hf_anom.msshf,levels=levels,cmap='seismic',extend='both',transform=projection)
# cbar = plt.colorbar(contourf, ax=ax, orientation='horizontal', pad=0.01,fraction=0.06, label="${Pa}$")

ax.gridlines(draw_labels=['left','bottom'],visible=False)

ax.coastlines()
ax.set_title("(a) Mean surface sensible heat flux - Pre-monsoon",fontsize=10)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.add_feature(cf.BORDERS.with_scale('10m'))

#jjas
# Data load
hf=xr.open_dataset(f'/content/drive/MyDrive/AP_HW/Scripts-ll/hf/data/lshf_events_anom-jjas.nc')

# Spatial region
# lat_range=slice(52,0)
# lon_range=slice(25,120)
lat_range=slice(27.5,20.00)
lon_range=slice(84,95.75)

hf=hf.sel(latitude=lat_range,longitude=lon_range)

hf_anom=hf.mean(dim='event_no',skipna=True)

latitude_hf=hf_anom['latitude'].values
longitude_hf=hf_anom['longitude'].values

# ,levels=levels
# mslhf plot
ax=axes[1,1]
contourf=ax.contourf(longitude_hf,latitude_hf,hf_anom.mslhf,levels=levels,cmap='seismic',extend='both',transform=projection)
# cbar = plt.colorbar(contourf, ax=ax, orientation='horizontal', pad=0.01,fraction=0.06, label="${Pa}$")

ax.gridlines(draw_labels=['bottom'],visible=False)

ax.coastlines()
ax.set_title("(d) Mean surface latent heat flux - Monsoon",fontsize=10)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.add_feature(cf.BORDERS.with_scale('10m'))
# msshf plot
ax=axes[1,0]
contourf=ax.contourf(longitude_hf,latitude_hf,hf_anom.msshf,levels=levels,cmap='seismic',extend='both',transform=projection)
# cbar = plt.colorbar(contourf, ax=ax, orientation='horizontal', pad=0.01,fraction=0.06, label="${Pa}$")


ax.gridlines(draw_labels=['left','bottom'],visible=False)

ax.coastlines()
ax.set_title("(c) Mean surface sensible heat flux - Monsoon",fontsize=10)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.add_feature(cf.BORDERS.with_scale('10m'))

# Create a single colorbar for both subplots
# Get the contour levels from the first subplot
cbar = plt.colorbar(contourf, ax=axes, orientation='horizontal', pad=0.01, fraction=0.02, aspect=35, label="${W} {m}^{-2}$")
cbar.set_ticks(levels[::1])  # Set the colorbar ticks to match the levels used in contour plots
# Adjust colorbar position if needed
cbar.ax.set_position([0.21, 0.075, 0.6, 0.025]) # [left, bottom, width, height]
# plt.tight_layout()
plt.subplots_adjust( wspace=0.05,hspace=-0.825,top=0.9)
plt.suptitle('Composite of Latent and Sensible Heat Flux Anomalies ', x=0.5, y=0.34,fontsize=11)
plt.show()

# Save the figure
fig.savefig("/content/drive/MyDrive/AP_HW/Scripts-ll/Manuscript codes/All_Figures/Fig_7_composite_of_lhf_and_shf_anomaly.jpg",
            dpi=300, format="jpg", bbox_inches="tight")

In [ ]:
print('executed all the lines')